In [5]:
with open('Sell_Short_Trades.txt') as f:
    content = f.read()
content[:2000]

'                                                                                                                \n DOTC                                                                                 RUN DATE : 06/14/17 21:56 \n                                                                                             RPT DATE :06/14/17 \n                                    TTS0126:SELL SHORT TRADES & SHORT EXEMPT                                    \n Symbol   Side    Cxl      Qty    Price       Bid        Ask        T-DatS-DatTradeID      TradeTiSS      Exbkr \n                                                                                                                \n TradeCommType  SourceCommission   Account   OrderID      GTL               Trailer Info               Clr      \n CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS        \n                                                                    /17  /17  256945   

In [7]:
split_lines = content.split("\n")
split_lines

['                                                                                                                ',
 ' DOTC                                                                                 RUN DATE : 06/14/17 21:56 ',
 '                                                                                             RPT DATE :06/14/17 ',
 '                                    TTS0126:SELL SHORT TRADES & SHORT EXEMPT                                    ',
 ' Symbol   Side    Cxl      Qty    Price       Bid        Ask        T-DatS-DatTradeID      TradeTiSS      Exbkr ',
 '                                                                                                                ',
 ' TradeCommType  SourceCommission   Account   OrderID      GTL               Trailer Info               Clr      ',
 ' CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS        ',
 '                                                              